In [1]:
import numpy as np
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import h5py
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [3]:
h5f = h5py.File('SVHN_single_grey1-2.h5', 'r')
print(list(h5f.keys()))

# Load the training, test and validation set
X_test = h5f['X_test'][:]
X_train = h5f['X_train'][:]
X_val = h5f['X_val'][:]
y_test = h5f['y_test'][:]
y_train = h5f['y_train'][:]
y_val = h5f['y_val'][:]

# Close this file
h5f.close()

#to be noted that this is a grey matrix

['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']


In [4]:
# this matrix is already split into train test and val. Plus an even matrix. Plus has 32, 32, 1 
print("X_train shape: ", X_train.shape, "y_train shape: ", y_train.shape)

X_train shape:  (42000, 32, 32) y_train shape:  (42000,)


In [5]:
img_size=32
num_labels=10
num_channel=1

def chg_format(dataset, labels):
    dataset=dataset.reshape((-1,img_size,img_size,num_channel)).astype(np.float32)
    labels=tf.keras.utils.to_categorical(labels, num_labels).astype(np.int32)
    return dataset, labels

In [6]:
X_train, y_train = chg_format(X_train, y_train)
X_test, y_test = chg_format(X_test, y_test)
X_val, y_val = chg_format(X_val, y_val)

print('Training set shape : ', X_train.shape, y_train.shape)
print('Validation set shape : ', X_val.shape, y_val.shape)
print('Test set shape : ', X_test.shape, y_test.shape)

Training set shape :  (42000, 32, 32, 1) (42000, 10)
Validation set shape :  (60000, 32, 32, 1) (60000, 10)
Test set shape :  (18000, 32, 32, 1) (18000, 10)


In [7]:
model = tf.keras.Sequential()
# Must define the input shape in the first layer of the neural network
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(32,32,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
# Take a look at the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        8224      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0

In [ ]:
#tf.keras.utils.to_categorical(labels, num_labels).astype(np.int32)

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, batch_size=64, epochs=11, validation_data=(X_val, y_val))

Train on 42000 samples, validate on 60000 samples
Epoch 1/11
42000/42000 [==============================] - 149s 4ms/sample - loss: 2.9007 - accuracy: 0.0979 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 2/11
42000/42000 [==============================] - 137s 3ms/sample - loss: 2.3031 - accuracy: 0.1007 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 3/11
42000/42000 [==============================] - 140s 3ms/sample - loss: 2.3028 - accuracy: 0.0985 - val_loss: 2.3027 - val_accuracy: 0.1000
Epoch 4/11
42000/42000 [==============================] - 141s 3ms/sample - loss: 2.3024 - accuracy: 0.1019 - val_loss: 2.2973 - val_accuracy: 0.1109
Epoch 5/11
42000/42000 [==============================] - 138s 3ms/sample - loss: 2.2983 - accuracy: 0.1045 - val_loss: 2.2892 - val_accuracy: 0.1244
Epoch 6/11
42000/42000 [==============================] - 137s 3ms/sample - loss: 2.2845 - accuracy: 0.1090 - val_loss: 2.2722 - val_accuracy: 0.1335
Epoch 7/11
42000/42000 [==========================

In [10]:
# Evaluate the model on test set
score = model.evaluate(X_test, y_test, verbose=0)
# Print test accuracy
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.7957778


In [ ]:
#IMPLEMENTING BATCH NORMALIOSATION

In [11]:
# Initialize Sequential model

def model_seq():
    model = tf.keras.models.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30, input_dim=13, activation='relu'),
    tf.keras.layers.Dense(40, input_dim=15, activation='relu'),
    tf.keras.layers.Dense(1, input_dim=18, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
#Accuracy matrix

In [18]:
y_pred=model.predict(X_test)

In [20]:
from sklearn.metrics import accuracy_score
score = (accuracy_score(y_test, y_pred)*100)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [12]:
#Confusion matrix

In [13]:
from sklearn.metrics import confusion_matrix

In [17]:
print("\n***Confusion matrix***\n",confusion_matrix(y_test,y_pred.round()))

ValueError: multilabel-indicator is not supported